In [1]:
import numpy as np
from numpy.random import uniform as u

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
open('/content/gdrive/My Drive/pagerank.txt','r').read()

'4 5\n5 8\n8 2\n8 8\n9 4\n9 1\n7 2\n5 9\n5 5\n7 7\n2 7\n4 1\n6 6\n3 8\n9 9\n4 8\n5 5\n2 4\n4 3\n1 8\n6 4\n8 4\n6 2\n3 8\n7 1\n9 6\n9 8\n2 7\n3 7\n4 9\n1 4\n3 4\n1 3\n3 6\n2 5\n1 2\n5 6\n4 6\n1 1\n6 4\n5 1\n8 6\n3 4\n9 9\n5 6\n4 5\n6 6\n6 7\n5 2\n6 8\n'

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [8]:
import json

In [24]:
path = '/content/gdrive/My Drive/pagerank.txt'

In [25]:
textR = sc.textFile(path)

In [26]:
textR.collect()

['4 5',
 '5 8',
 '8 2',
 '8 8',
 '9 4',
 '9 1',
 '7 2',
 '5 9',
 '5 5',
 '7 7',
 '2 7',
 '4 1',
 '6 6',
 '3 8',
 '9 9',
 '4 8',
 '5 5',
 '2 4',
 '4 3',
 '1 8',
 '6 4',
 '8 4',
 '6 2',
 '3 8',
 '7 1',
 '9 6',
 '9 8',
 '2 7',
 '3 7',
 '4 9',
 '1 4',
 '3 4',
 '1 3',
 '3 6',
 '2 5',
 '1 2',
 '5 6',
 '4 6',
 '1 1',
 '6 4',
 '5 1',
 '8 6',
 '3 4',
 '9 9',
 '5 6',
 '4 5',
 '6 6',
 '6 7',
 '5 2',
 '6 8']

In [27]:
##1. Python list: Output sorted (based on the node id) list of incoming edges
keyValue = textR.map(lambda x: (x.split(" ")[0], x.split(" ")[1])).distinct().sortByKey()
keyValue.collect()

[('1', '8'),
 ('1', '4'),
 ('1', '1'),
 ('1', '3'),
 ('1', '2'),
 ('2', '7'),
 ('2', '5'),
 ('2', '4'),
 ('3', '7'),
 ('3', '6'),
 ('3', '8'),
 ('3', '4'),
 ('4', '1'),
 ('4', '8'),
 ('4', '9'),
 ('4', '5'),
 ('4', '3'),
 ('4', '6'),
 ('5', '5'),
 ('5', '6'),
 ('5', '2'),
 ('5', '8'),
 ('5', '9'),
 ('5', '1'),
 ('6', '6'),
 ('6', '2'),
 ('6', '7'),
 ('6', '4'),
 ('6', '8'),
 ('7', '2'),
 ('7', '7'),
 ('7', '1'),
 ('8', '8'),
 ('8', '4'),
 ('8', '2'),
 ('8', '6'),
 ('9', '4'),
 ('9', '1'),
 ('9', '9'),
 ('9', '8'),
 ('9', '6')]

In [30]:
##2. Python dictionary: Show only the nodes who has exactly 5 incoming edges
nodes_list = keyValue.groupByKey().filter(lambda l: len(l[1])==5)
print("nodes having 5 incoming edeges: ", nodes_list.map(lambda x:(x[0], list(x[1]))).collectAsMap())

nodes having 5 incoming edeges:  {'1': ['8', '4', '1', '3', '2'], '9': ['4', '1', '9', '8', '6'], '6': ['6', '2', '7', '4', '8']}


In [35]:
##3. Python list: show the incoming edges of the node ‘1’
print("The incoming edges of Node 1 are : \n",nodes_list.filter(lambda x :'1' in x[0]).mapValues(list).collect())

The incoming edges of Node 1 are : 
 [('1', ['8', '4', '1', '3', '2'])]
